In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
Datadirectory = r"C:\Users\Aditya Venkatesh\Drowsiness detection new\archive (6)\mrleyedataset"
Classes = ["Close-Eyes", "Open-Eyes"]
for category in Classes:
    path = os.path.join(Datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        backtorgb = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
        plt.imshow(img_array, cmap="gray")
        plt.show()
        break
    break

In [ ]:
img_size = 24
new_array = cv2.resize(backtorgb, (img_size, img_size))
plt.imshow(new_array, cmap="gray")
plt.show()

In [ ]:
training_Data = []
def create_training_Data():
   for category in Classes:
       path = os.path.join(Datadirectory, category)
       class_num = Classes.index(category) # 0 1,
       for img in os.listdir(path):
           try:
               img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
               backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB)
               new_array = cv2.resize(backtorgb, (img_size, img_size))
               training_Data.append([new_array,class_num])
           except Exception as e:
               pass
create_training_Data()

In [ ]:
import random
random.shuffle(training_Data)

In [ ]:
X = []
y = []
for features,label in training_Data:
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 3)

In [ ]:
y

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping

In [ ]:
# Convert X and y to NumPy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)

model.save('Mymodel.h5', overwrite=True)

In [ ]:
!pip install pygame

In [ ]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
new_model = load_model(r"C:\Users\Aditya Venkatesh\Downloads\Drowsiness detection\Drowsiness detection\models\cnnCat2.h5")

In [ ]:
label=["Close_eye","Open_eye"]

In [ ]:
import winsound
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Initialize sound mixer
winsound.PlaySound(None, winsound.SND_PURGE)
frequency = 2500  # Set frequency to 2500
duration = 1500  # Set duration to 1500 ms == 1.5 sec

# Load Haar cascade classifiers for face and eye detection
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eyeCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)  # Start capturing video from webcam

# Check if webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

#cap.set(cv2.CAP_PROP_FPS, 5)  # Set FPS of video capture

counter = 0  # Counter to track consecutive frames with closed eyes

try:
    while True:
        ret, frame = cap.read()  # Read frame from video feed
        
        # Check if frame is None (failed to capture)
        if frame is None:
            print("Error: Failed to capture frame.")
            continue

        # Convert frame to grayscale for face and eye detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect faces in the frame
        faces = faceCascade.detectMultiScale(gray, 1.1, 4)
        
        # Loop through detected faces
        for (x, y, w, h) in faces:
            # Draw rectangle around face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Extract region of interest (ROI) for eyes
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            
            # Detect eyes within the face ROI
            eyes = eyeCascade.detectMultiScale(roi_gray)
            
            # Loop through detected eyes
            for (ex, ey, ew, eh) in eyes:
                # Draw rectangle around eyes
                cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
                
                # Extract region of interest (ROI) for each eye
                eye_roi = roi_gray[ey:ey+eh, ex:ex+ew]
                
                # Preprocess eye image for prediction
                eye_image = cv2.resize(eye_roi, (24, 24))
                eye_image = eye_image.reshape(24,24,-1)
                eye_image = np.expand_dims(eye_image, axis=0)
                eye_image = eye_image / 255.0
                
                # Make prediction using the model
                predictions = new_model.predict(eye_image)
                print(predictions)
                index=np.argmax(predictions[0])
                prediction_class= label[np.argmax(predictions[0])]
                
                # Determine if eyes are closed based on prediction
                if index == 1:
                    status = "Open-Eyes"
                    counter = 0  # Reset counter if eyes are open
                else:
                    status = "Close-Eyes"
                    counter += 1  # Increment counter if eyes are closed
                    
                    # Alert if eyes remain closed for consecutive frames
                    if counter > 10:
                        status = "Drowsiness Detected!"
                        winsound.Beep(frequency, duration)  # Sound alert
                        counter = 0  # Reset counter after alert
                        
                # Display status text
                cv2.putText(frame, status, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        # Display frame with annotations using Matplotlib
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title('Drowsiness Detection')
        plt.axis('off')  # Turn off axis
        plt.show()

except KeyboardInterrupt:
    print("Keyboard Interrupt. Exiting...")
finally:
    # Release video capture and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()
